In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd

from utils.loading import load_paths_and_graph

In [2]:
paths_and_graph = load_paths_and_graph()

loading raw data from tsv files...
formatting data...
extracting additional information from source code...


In [3]:
paths_and_graph["shortest-path-distance-matrix"].shape

(4604, 4604)

In [4]:
paths_and_graph["paths_finished"]

,hashedIpAddress,timestamp,durationInSec,path,rating
0,6a3701d319fc3754,1297740409,166,14th_century;15th_century;16th_century;Pacific...,NULL
1,3824310e536af032,1344753412,88,14th_century;Europe;Africa;Atlantic_slave_trad...,3
2,415612e93584d30e,1349298640,138,14th_century;Niger;Nigeria;British_Empire;Slav...,NULL
3,64dd5cd342e3780c,1265613925,37,14th_century;Renaissance;Ancient_Greece;Greece,NULL
4,015245d773376aab,1366730828,175,14th_century;Italy;Roman_Catholic_Church;HIV;R...,3
...,...,...,...,...,...
51313,15a13a1d66ef5456,1349231015,66,Yagan;Ancient_Egypt;Civilization,NULL
51314,2ef7ac844cefda58,1300254138,165,Yagan;Folklore;Brothers_Grimm;<;19th_century;C...,3
51315,12863abb7887f890,1385095372,228,Yagan;Australia;England;France;United_States;T...,NULL
51316,19f8284371753362,1298792567,56,Yarralumla%2C_Australian_Capital_Territory;Aus...,1


In [5]:
pd.DataFrame(
    data={
        "key": list(paths_and_graph.keys()),
        "shape": list(map(lambda d: d.shape, paths_and_graph.values()))
    }
)

,key,shape
0,shortest-path-distance-matrix,"(4604, 4604)"
1,paths_finished,"(51318, 5)"
2,articles,"(4541, 4)"
3,paths_unfinished,"(24875, 6)"
4,links,"(119882, 4)"
5,categories,"(5204, 3)"


In [6]:
paths_and_graph["links"]

,linkSource,linkTarget,linkSource_decoded,linkTarget_decoded
0,%C3%81ed%C3%A1n_mac_Gabr%C3%A1in,Bede,Áedán_mac_Gabráin,Bede
1,%C3%81ed%C3%A1n_mac_Gabr%C3%A1in,Columba,Áedán_mac_Gabráin,Columba
2,%C3%81ed%C3%A1n_mac_Gabr%C3%A1in,D%C3%A1l_Riata,Áedán_mac_Gabráin,Dál_Riata
3,%C3%81ed%C3%A1n_mac_Gabr%C3%A1in,Great_Britain,Áedán_mac_Gabráin,Great_Britain
4,%C3%81ed%C3%A1n_mac_Gabr%C3%A1in,Ireland,Áedán_mac_Gabráin,Ireland
...,...,...,...,...
119877,Zulu,South_Africa,Zulu,South_Africa
119878,Zulu,Swaziland,Zulu,Swaziland
119879,Zulu,United_Kingdom,Zulu,United_Kingdom
119880,Zulu,Zambia,Zulu,Zambia


In [7]:
articles = paths_and_graph["articles"]
articles.sort_values(by="num_links", ascending=False)

,article,article_decoded,links,num_links
3920,Sudan,Sudan,"[(2007 Schools Wikipedia Selection, index, ind...",606
1433,Europe,Europe,"[(2007 Schools Wikipedia Selection, index, ind...",558
2504,List_of_countries,List_of_countries,"[(2007 Schools Wikipedia Selection, index, ind...",558
1694,Germany,Germany,"[(2007 Schools Wikipedia Selection, index, ind...",552
2863,Mozambique,Mozambique,"[(2007 Schools Wikipedia Selection, index, ind...",550
...,...,...,...,...
2351,Klinefelter%27s_syndrome,Klinefelter's_syndrome,"[(2007 Schools Wikipedia Selection, index, ind...",4
4545,Wowpurchase,Wowpurchase,"[(place your order by post, Contact, Contact),...",3
1600,Friend_Directdebit,Friend_Directdebit,[(use this link to making a smaller regular do...,3
1210,Directdebit,Directdebit,"[(children charity, Children_Charity, Children...",2


In [8]:
relations = pd.concat(
    list(map(lambda row: pd.DataFrame(
        data={
            "from": [row.article_decoded]*len(row.links),
            "to": [link[2] for link in row.links]
        }
    ), articles.itertuples(index=False)))
)

relations.drop_duplicates(inplace=True)
relations

,from,to
0,Áedán_mac_Gabráin,index
1,Áedán_mac_Gabráin,subject.History.British_History.British_Histor...
2,Áedán_mac_Gabráin,subject.People.Historical_figures
3,Áedán_mac_Gabráin,4250.png
5,Áedán_mac_Gabráin,D%C3%A1l_Riata
...,...,...
16,Zulu,Cape_Town
17,Zulu,AK-47
18,Zulu,Ladysmith_Black_Mambazo
23,Zulu,Wikipedia_Text_of_the_GNU_Free_Documentation_L...


In [9]:
relations.groupby(by="from").count().sort_values("to", ascending=False).head(5)

,to
from,
List_of_countries,491
List_of_circulating_currencies,471
List_of_sovereign_states,423
United_States,334
Africa,288


In [10]:
relations.groupby(by="to").count().sort_values("from", ascending=False).head(15)

,from
to,
index,4536
Wikipedia_Text_of_the_GNU_Free_Documentation_License,4536
disclaimer,4536
United_States,1519
United_Kingdom,962
France,947
Europe,919
England,734
Germany,733


In [11]:
relations.query("`from` == `to`")

,from,to
24,American_Revolutionary_War,American_Revolutionary_War
10,Athens,Athens
63,Autostereogram,Autostereogram
42,Baltic_Sea,Baltic_Sea
13,Beijing,Beijing
...,...,...
10,War,War
4,Wikimedia_Foundation,Wikimedia_Foundation
3,Wikipedia_Text_of_the_GNU_Free_Documentation_L...,Wikipedia_Text_of_the_GNU_Free_Documentation_L...
84,World_Trade_Organization,World_Trade_Organization
